In [1]:
import csv
import numpy as np
import math
import random
from geneticalgorithm2 import geneticalgorithm2 as ga 
from scipy.spatial.distance import cdist
import pandas as pd
import math
import heapq
np.warnings.filterwarnings('ignore', category=np.VisibleDeprecationWarning)  

In [2]:

shores = np.loadtxt("shore.csv", dtype='float', delimiter=',')
elevations = np.loadtxt("elevation.csv", dtype='float', delimiter=',')

# shores = shores - np.amin(shores)
elevations = np.abs(elevations)
# elevations = elevations - np.amin(elevations)

In [3]:
def pareto_crowds(pop: np.ndarray):

    #Create ranks based on domination
    #   Iterate through the population and get candidates such that tehre are no other candidates with better objective values for BOTH functions (can exist 1)
    #   These candidates are given smallest current rank and removed from population to repeat and rank all others
    popN = pop.shape[0]


    #Dictionary of all candidates "dominated" by given index candidate
    dominated_sets = {}
    #Dictionary of all candidates in rank order
    rank_sets = {key: [] for key in range(1,popN+1,1)}

    ranks = [popN]*popN
    crowdings = [0]*popN
    F = []

    #Go through population for each candidate and check whether there exists candidates that (1) it dominates or (2) is dominated by
    for p in range(popN):
        sp = []
        dnp = 0
        for q in range(popN):
            if pop[p,1] < pop[q,1] and pop[p,2] < pop[q,2]: #p dominates q
                sp.append(q)
            elif pop[q,1] < pop[p,1] and pop[q,2] < pop[p,2]: #q dominates p
                dnp = dnp + 1

        #There are no candidate taht dominate given candidate, it is given rank 1
        if dnp == 0:
            ranks[p] = 1 
            rank_sets[1] = rank_sets[1] + [p]
            F.append(p)
        dominated_sets[p] = [sp,dnp] 


    i = 1
    #Iterate through all top rank candidate and find next candidates FROM their lists of dominations (now these are next highest rank candidates)
    while (len(F) != 0):
        Q = []
        for p in F:

            for q in dominated_sets[p][0]:
                dominated_sets[q][1] = dominated_sets[q][1] - 1
                #Removed all current rank candidates from given candidate's dominated by list, since it is 0 it is among best next rank
                if dominated_sets[q][1] == 0:
                    ranks[q] = i + 1
                    rank_sets[i + 1] = rank_sets[i + 1] + [q]
                    Q.append(q)
        i = i + 1
        F = Q

    #objs = ["fitness_func","cable_total"]

    #Get crowding distance by getting distance to neighbors of given candidate (distances are proportional to min/max of given objective)
    for r in range(1,i,1):
        l = len(rank_sets[r])
        arr = np.array(rank_sets[r])        
        for o in range(2):            
            #sort by obj            
            sort_arr = arr[np.argsort(pop[:,o+1][arr])]

            #set crowd for 1 and final to 10000 (these are expected to contain more information and will be chosen if required)
            crowdings[sort_arr[0]] = -1000000
            crowdings[sort_arr[l-1]] = -1000000

            #Get min/max and get distance prortion to their difference
            o_min = np.copy(pop[sort_arr[0],o+1])
            o_max = np.copy(pop[sort_arr[l-1],o+1])            
            for n in range(1,l-1):
                if o_min == o_max:
                    crowdings[sort_arr[n]] = crowdings[sort_arr[n]] + 0
                elif crowdings[sort_arr[n]] != -1000000:
                    #subtract since we are later sorting ascending 
                    crowdings[sort_arr[n]] = crowdings[sort_arr[n]]  -  ((pop[sort_arr[n-1],o+1] + pop[sort_arr[n+1],o+1]) / (o_max - o_min) )
    #use the ranks to get crowding distnaces
    return [ranks,crowdings]

In [4]:
def mixed_func(pop):
    popN = pop.shape[0]
    ranks = [popN]*popN
    #Crowding not necessary, returned for consistency
    crowdings = [0]*popN

    #rank based on both wake(turbine costs) and cable length
    #Get the min,max for both objectives and rank based on the standing of given candidate in the population
    min_wake = np.min(pop[:,1])
    max_wake = np.max(pop[:,1])

    min_cable = np.min(pop[:,2])
    max_cable = np.max(pop[:,2])

    costs = [0] * popN
    for p in range(popN):
        #arbitrary weights 75% and 25% respectively
        costs[p] = (0.75*( ( pop[p,1] - min_wake ) / (max_wake-min_wake))) + (0.25*( (pop[p,2] - min_cable )/(max_cable-min_cable)))

    #Ranks are sort order for new combination values
    ranks = np.argsort(costs)
    return [ranks,crowdings]

In [5]:

def Prim_mst(nodes,edges,center):
    #Implemented based on CodeSavant's implementation for my context
    nodes_mst = set()

    mst_costs = 0
    nodesN = len(nodes)

    #Edges stored ad dict
    edges_all = { tuple(key): [] for key in nodes}
    
    #add all nodes to heap
    for [s,e,c] in edges:
        heapq.heappush(edges_all[tuple(s)],(c,tuple(e)))
        heapq.heappush(edges_all[tuple(e)],(c,tuple(s)))

    cost, dest = 0,1
    while len(nodes_mst) < nodesN:
        nodes_smallest = tuple(center)

        #Go through nodes to get the node that is connected with the next edge
        for n in nodes_mst:
            while len(edges_all[n]) > 0 and edges_all[n][0][dest] in nodes_mst:
                heapq.heappop(edges_all[n])
            
            #no end nodes from given node are outside of MST
            if len(edges_all[n]) == 0:
                continue
            
            #nodes found
            if len(edges_all[nodes_smallest]) == 0 or edges_all[n][0][cost] < edges_all[nodes_smallest][0][cost]:
                nodes_smallest = n

        #Pop from heap (lowest one present at top) to get the correct edge with smallest weight
        to_add = heapq.heappop(edges_all[nodes_smallest])

        #add cost to MST total (distance + elevation values of each turbine - connection from bottom of sea to turbine)
        mst_costs += to_add[cost]
        mst_costs += elevations[to_add[dest][0],to_add[dest][1]]
        mst_costs += elevations[nodes_smallest[0],nodes_smallest[1]]

        #add the nodes to the MST (handles duplicates)
        nodes_mst.add(nodes_smallest)
        nodes_mst.add(to_add[dest])
    return mst_costs

In [6]:
def get_offshore_dist(center):
    #return distance to shore in m
    return shores[center[0],center[1]]



def get_array_dist(xes,yes,center):
    #Uses MST to give array cable length

    #Create arrray of nodes
    nodes = [[x,y] for x,y in zip(xes,yes)]
    if center not in nodes:
        nodes.append(center)

    #creates edges (the graph will start out as complete) [start,end,length] 
    pairs = [(a, b) for idx, a in enumerate(nodes) for b in nodes[idx + 1:]]
    edges = []
    for (s,e) in pairs:
        edges.append([s,e, 140*math.dist([s[0], s[1]], [e[0], e[1]]) ])
    

    dist = Prim_mst(nodes,edges,center)
    return dist

def cable_total(ones):
    xes = [i%50 for i in ones]
    yes = [int(i/50) for i in ones]

    #calculate the centerpoint and corresponding distance to shore
    center = [ int(np.mean(xes)), int(np.mean(yes)) ]
    export = get_offshore_dist(center)

    #calculate interconnection of the turbines
    array = get_array_dist(xes,yes,center)

    #return the total distance values
    return export + array

In [7]:
interactions = []
#import all the interaction matricies created beforehand and add to array (flattened versions of the matrices)
for i in range(50):
    for j in range(50):
        mat = np.loadtxt(open("interactionMats/"+str(i)+"_"+str(j)+".csv", "rb"), delimiter=",", skiprows=0)
        arr = np.array(mat).flatten()

        interactions.append(arr)



In [8]:

def fitness_func(ones):
    #add all the interaction values
    wake = 0
    if len(ones) > 50:
        return 1000000

    for i in ones:
        tot = 0
        for j in ones:
            tot += interactions[i][j]
        wake += tot
    return wake
def make_arr(ones: np.ndarray):
    arr = [0] * 2500
    for i in ones:
        arr[i] = 1
    return np.array(arr)
def make_random(number, turbine,fitnessT):
    pop = []
    for i in range(number):
        turbines = np.random.choice(np.arange(0, 2500), turbine, replace = False)
        binary =  make_arr(turbines)

        wake = fitness_func(turbines)
        cable = cable_total(turbines)
        pop.append([binary,wake,cable,0,0])
    
    #pareto ranks
    pop = np.array(pop)
    paretos = globals()[fitnessT](pop)
    #crowding distances
    new_pop = np.c_[pop[:,0],pop[:,1],pop[:,2],paretos[0],paretos[1]]
    return new_pop
def sort_tot(pop):
    #sort by the parent then order by wake values
    return pop[np.lexsort((pop[:,4],pop[:,3]))]

In [9]:
def rand_mutation(bitstring, prob):
   
    if np.random.rand() < prob:
        lens = len(bitstring)
        i = int(np.random.rand() * lens)

        # flip the bit and one with opposite value (to keep same number of turbines)
        prev = np.copy(bitstring[i])
        opp = np.where(bitstring == (1-prev))[0]
        bitstring[i] = 1 - np.copy(prev)

        bitstring[  opp[int(np.random.rand()*len(opp))]  ] = np.copy(prev)
    return bitstring

def uniform_crossover(x: np.ndarray, y: np.ndarray):
    #randomly selects 1's from either sides
    x1 = np.where(x == 1)[0]
    y1 = np.where(y == 1)[0]

    ofs1 = []
    ofs2 = []
    
    all = list(set(x1) | set(y1))

    ofs1 = random.sample(all,50)
    ofs2 = random.sample(all,50)


    ofs1 = make_arr(ofs1)
    ofs2 = make_arr(ofs2)    
    return [ofs1, ofs2]


def tournament_selection(pop):
    #ramdomly select 4 parents and pick 2 in each block according to thier fitness
    pairs = [] #indexes of parents 
    scores = pop[:,1]

    #get random selections (size of population
    parents = np.random.choice(a=np.arange(0,scores.size),size=2*scores.size,replace=True)

        
    for i in range(int(parents.size/4)):
        #pick parent 1 based on pareto (or if same using crowding dist)
        par1 = 0
        if pop[parents[(i*4)],3] == pop[parents[(i*4)+1],3]:
            #same pareto rank
            par1 = np.argsort(pop[parents[i*4:(i*4)+2],4])[0]
        else:
            par1 = np.argsort(pop[parents[i*4:(i*4)+2],3])[0]
        
        #pick parent 2 based on pareto (or if same using crowding dist)
        par2 = 2
        if pop[parents[(i*4)+2],3] == pop[parents[(i*4)+3],3]:
            #same pareto rank
            par2 = 2+ np.argsort(pop[parents[(i*4)+2:(i*4)+4],4])[0]
        else:
            par2 = 2+ np.argsort(pop[parents[(i*4)+2:(i*4)+4],3])[0]
        
        pairs.append([pop[parents[(i*4)+par1]][0],pop[parents[(i*4)+par2]][0]])

    return pairs


In [10]:
def multi_ga(popN,fitnessT,iter):
    #We create same number of children as parents for NSGA2
    pop =  make_random(popN,50,fitnessT)
    pop = sort_tot(pop)

    prob_mutation = 0.1 #probability of a child going through mutation
    for it in range(iter):
        #We default to tournament selection since NSGA2 calls for it
        parent_pairs = tournament_selection(pop)
        children = []
        for [a,b] in parent_pairs:

            #Create children using uniform crossover and rand mutation
            child_pair = uniform_crossover(a,b)
            c1_bin = rand_mutation(child_pair[0],prob_mutation)
            c2_bin = rand_mutation(child_pair[1],prob_mutation)
            
            #Get the turbine positions to calcualte objective values
            ones_c1 = []
            ones_c2 = []
            for i in range(2500):
                if c1_bin[i] == 1:
                    ones_c1.append(i)
                if c2_bin[i] == 1:
                    ones_c2.append(i)

            #add childred to population
            children.append([c1_bin,fitness_func(ones_c1),cable_total(ones_c1),0,0])
            children.append([c2_bin,fitness_func(ones_c2),cable_total(ones_c2),0,0])

        children = np.array(children)
        total_pop = np.concatenate((children, pop))
        
        #get pareto ranks and crowding distances for next pop 
        paretos = globals()[fitnessT](total_pop)
        next_pop = np.c_[total_pop[:,0],total_pop[:,1],total_pop[:,2],paretos[0],paretos[1]]
        
        #remake population with 50/50 split old and new - elitist 
        pop = sort_tot(next_pop)
        pop = pop[:popN,]

    #Best candidate is in rank 1
    rank1s = [p for p in pop if p[3]==1]
    #I decided to go with best wake performing candidate as best candidate
    rank1s = sorted(rank1s, key=lambda x: x[1])
    return rank1s[0]

In [45]:
# final = multi_ga(1000,1000)

#best = multi_ga(100,"pareto_crowds",100)

In [44]:
# ones = []
# for i in range(2500):
#     if my_data2[i] == 1:
#         ones.append(i)

# xes = [i%50 for i in ones]
# yes = [int(i/50) for i in ones]

#calculate the centerpoint and corresponding distance to shore
# center = [ int(np.mean(xes)), int(np.mean(yes)) ]
# nodes = [[x,y] for x,y in zip(xes,yes)]
# if center not in nodes:
#     nodes.append(center)
